# Final Project: Huggingface Transformers를 이용한 한-영 번역

### 개요
- 한-영 번역 프로그램을 Huggingface의 Transformers와 Dataset을 이용해서 필요한 크기로 데이터를 줄이고 이를 모델에서 파인튜닝하여 번역 성능을 살펴보고 실제 예문의 번역을 출력
- [Huggingface NLP course의 7장 Translation](https://huggingface.co/learn/nlp-course/chapter7/4?fw=pt)을 근간으로 해서 (거의 그대로 활용할 수 있음) 구현할 수 있음
- Dataset을 자료를 받아서 필요한 크기로 나누고, 학습에 필요한 형태로 Dataset을 재구조화하고 tokenize하는 모듈을 구현
- 공개된 자료를 바탕으로 구현하기 때문에 성능보다는 전체 번역모듈을 Huggingface로 구현해보는 것을 주목표로 하기 때문에 완결성이 있어야 하며, 실제로 작동해야 함.
- 파일 이름 NLPProject_이름/그룹.ipynb
- Due 3월 9일 11시 59분

## 필요한 모듈 설치
- 프로그램 실행에 필요한 모듈, Huggingface, Dataset 등을 각자 알아서 설치

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
# T5 모델이 sentence piece tokenizer를 사용하기 때문에 데이터를 이 형태로 토크나이즈 하기 위해
!pip install sentencepiece

In [ ]:
!pip install accelerate

In [ ]:
# 모델 평가를 위해
!pip install evaluate

In [ ]:
#bleu 스코어 계산을 위해
!pip install sacrebleu

## Dataset
- Huggingface Hub에 있는 Dataset 중 `bongsoo/news_talk_en_ko` 는 한국어-영어뉴스 기사를 병렬로 작성한 130만 개의 데이터 셋이다.
- 이 데이터셋을 읽어서 colab에서 돌릴 수 있게, training, validation, test 데이터로 각각 120,000, 9,000, 1,000으로 줄여서 학습에 필요한 구조로 만듬
- 데이터를 자를때 순차적으로 자르지 말고 전체 데이터를 셔플한 후 필요한 크기로 자를 것
- 데이터셋을 pandas 형식으로 받은 후 할 수도 있고 여러 가능한 방법이 있음

In [1]:
from datasets import load_dataset

en_ko = load_dataset("bongsoo/news_talk_en_ko")

In [2]:
en_ko

DatasetDict({
    train: Dataset({
        features: ["Skinner's reward is mostly eye-watering.", '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.'],
        num_rows: 1299999
    })
})

## Huggingface
- 학습에 필요한 Huggingface 모델 사용
- AutoTokenizer, AutoModelForSeq2SeqLM 등을 사용
- 학습에 사용할 모델은 [T5](https://github.com/AIRC-KETI/ke-t5)("KETI-AIR/ke-t5-base")를 사용할 것
- T5모델은 트랜스포머의 인코더, 디코더 구조를 모두 사용하는 모델로 번역기를 만들 때 사용할 수 있는 모델이다. 
- 아래처럼 모델 체크 포인트와 T5 모델에 입력될 최대 토큰 길이를 적절히 설정한다.

In [ ]:
model_ckpt = "KETI-AIR/ke-t5-base"
max_token_length = 64

## Tokenizer
- T5는 sentencepiece tokenizer를 사용하기 때문에 한-영 병렬 데이터의 자료를 학습시키기 위해서는 이 데이터를 tokenizer를 써서 프로세싱을 해야 한다. 이를 위한 모듈을 만들고 한국어, 영어데이터를 tokenize하여 모델에 입력할 수 있는 형태로(tokenized-dataset) 바꾼다
- 이를 위해서 Dataset의 map()을 활용하도록 한다.

## Model
- 학습에 필요한 모델 설정

## Collator
- 학습할 자료를 정렬하고 모델에 배치 단위로 넘겨주기 위해 준비

## Metric
- 학습한 모델을 측정할 매트릭을 준비
- 번역 모델에서는 주로 BLEU 점수를 사용
- BLEU 점수는 번역기가 생성한 문장이 레퍼런스(정답이라는 표현을 사용하지 않는 이유는 제대로 된 번역 문장이 오직 하나가 아니기 때문)문장과 얼마나 비슷한지 측정하는 점수

- sacrebleu 라이브러리는 BLEU 구현체에서 사실상 표준 라이브러리이며 각 모델이 다른 토크나이저를 쓰는 경우 이를 BPE로 통일 시켜 BLEU 점수를 계산



## 모델 학습(Train)
- 학습을 간단히 하기위해 허깅페이스에서 제공하는 Seq2SeqTrainer클래스와 학습 세부 조건은 Seq2SeqTrainingArguments를 활용할 수 있으나, 본 과제에서는 이를 쓰지 말고 Training를 직접 구현하도록 한다. Dataloader, Scheduler, ACCELERATOR, Optimizer 등을 설정하고 실제로 training loop를 돌려서 학습하고, evaluation 데이터로 성능을 검증
- colab에서 돌리기 위해서는 성능이 저하되겠지만, batch size 등을 적당하게 설정해야 함.

## 모델 테스트 (Test)
- 학습된 모델을 가지고 테스트 데이터로 테스트

## Inference
- Assignment2에 쓰였던 문장들을 이 학습된 모델에서 그 결과를 살펴 보아라

- 모든 액체, 젤, 에어로졸 등은 1커트짜리 여닫이 투명봉지 하나에 넣어야 합니다.
- 미안하지만, 뒷쪽 아이들의 떠드는 소리가 커서, 광화문으로 가고 싶은데 표를 바꾸어 주시겠어요?
- 은행이 너무 멀어서 안되겠네요. 현찰이 필요하면 돈을 훔시세요
- 아무래도 분실한 것 같으니 분실 신고서를 작성해야 하겠습니다. 사무실로 같이 가실까요?
- 부산에서 코로나 확진자가 급증해서 병상이 부족해지자 확진자 20명을 대구로 이송한다
- 변기가 막혔습니다
- 그 바지 좀 보여주십시오. 이거 얼마에 살 수 있는 것 입니까?
- 비가 와서 백화점으로 가지 말고 두타로 갔으면 좋겠습니다.
- 속이 안좋을 때는 죽이나 미음으로 아침을 대신합니다
- 문대통령은 집단 이익에서 벗어나라고 말했다
- 이것 좀 먹어 볼 몇 일 간의 시간을 주세요
- 이 날 개미군단은 외인의 물량을 모두 받아 내었다
- 통합 우승의 목표를 달성한 NC 다이노스 나성범이 메이저리그 진출이라는 또 다른 꿈을 향해 나아간다
- 이번 구조 조정이 제품을 효과적으로 개발 하고 판매 하기 위한 회사의 능력 강화 조처임을 이해해 주시리라 생각합니다
- 요즘 이 프로그램 녹화하며 많은 걸 느낀다